In [55]:
import pandas as pd
import datetime

In [56]:
#json einlesen
df_daten = pd.read_json("dump-stg-api.json", orient="records")
print(df_daten)

                                     _id  \
0   {'$oid': '6123b80df66ddb7127c2abfd'}   
1   {'$oid': '612ca185f66ddb4497c2abfe'}   
2   {'$oid': '6130bb10f66ddb7ebdc2abff'}   
3   {'$oid': '6130c0d5f66ddb9bf3c2ac00'}   
4   {'$oid': '6144421471a7cc0acf1a3bd1'}   
..                                   ...   
67  {'$oid': '6204c6c41c2f8b688bef4b29'}   
68  {'$oid': '6204c8811c2f8ba5d1ef4b2a'}   
69  {'$oid': '6204d5011c2f8b4689ef4b2b'}   
70  {'$oid': '6204d64d1c2f8bbb93ef4b2c'}   
71  {'$oid': '6205117f1c2f8b6e27ef4b2d'}   

                                              history  
0   {'task-000': [{'correct': False, 'date': 16297...  
1   {'task-000': [{'correct': True, 'date': 163031...  
2   {'task-000': [{'correct': True, 'date': 163058...  
3   {'task-000': [{'correct': True, 'date': 163058...  
4                                                 NaN  
..                                                ...  
67  {'task-000': [{'correct': True, 'date': 164448...  
68  {'task-000': [{'cor

In [57]:
df_daten_clean = df_daten

In [58]:
df_na_rows = df_daten_clean.isna()

In [59]:
df_daten_clean

,_id,history
0,{'$oid': '6123b80df66ddb7127c2abfd'},"{'task-000': [{'correct': False, 'date': 16297..."
1,{'$oid': '612ca185f66ddb4497c2abfe'},"{'task-000': [{'correct': True, 'date': 163031..."
2,{'$oid': '6130bb10f66ddb7ebdc2abff'},"{'task-000': [{'correct': True, 'date': 163058..."
3,{'$oid': '6130c0d5f66ddb9bf3c2ac00'},"{'task-000': [{'correct': True, 'date': 163058..."
4,{'$oid': '6144421471a7cc0acf1a3bd1'},NaN
...,...,...
67,{'$oid': '6204c6c41c2f8b688bef4b29'},"{'task-000': [{'correct': True, 'date': 164448..."
68,{'$oid': '6204c8811c2f8ba5d1ef4b2a'},"{'task-000': [{'correct': True, 'date': 164448..."
69,{'$oid': '6204d5011c2f8b4689ef4b2b'},"{'task-000': [{'correct': True, 'date': 164448..."
70,{'$oid': '6204d64d1c2f8bbb93ef4b2c'},"{'task-000': [{'correct': False, 'date': 16444..."


In [60]:
#ID $oid entfernen
for i in range(len(df_daten_clean)):
    df_daten_clean["_id"][i] = df_daten_clean["_id"][i]["$oid"]

In [61]:
#Columns Dataframe
df_structure = pd.DataFrame(columns=["log_id", "correct", "query", "date1", "task"])

In [62]:
df_structure

,log_id,correct,query,date1,task


In [63]:
#Json in Dataframe umwandeln
index_value = 0
for i in range(len(df_daten_clean)):
    print(i)
    if df_na_rows["history"][i]:
        pass
    else:
        split = df_daten_clean["history"][i]
        print(split)
        keys = split.keys()
        for n in keys:
            for k in range(len(split[n])):
                index_value += 1
                dict_copy = split[n][k]
                if "\n" in dict_copy["query"] or "\t" in dict_copy["query"]: #Newline in query und tab in query entfernen für die Datenbank
                    dict_copy["query"] = dict_copy["query"].replace("\n", " ")
                    dict_copy["query"] = dict_copy["query"].replace("\t", "")
                new_line = pd.DataFrame({"correct": dict_copy["correct"], "query": dict_copy["query"] , "date1": dict_copy["date"], "task": n, "log_id": df_daten_clean["_id"][i]}, index=[index_value])
                df_structure = df_structure.append(new_line) #append to dataframe

0
{'task-000': [{'correct': False, 'date': 1629799177614, 'query': 'from ghosts SELECT name'}, {'correct': True, 'date': 1629799183760, 'query': 'SELECT name from ghosts'}], 'task-086': [{'correct': True, 'date': 1641556069420, 'query': "SELECT north_zone.squaremeter/count(*)\n\tFROM (SELECT zone, sum(squaremeter) squaremeter\n\t\tFROM zoo_cage\n\t\tWHERE zone = 'north'\n\t\tGROUP BY zone) north_zone INNER JOIN \n\t\tzoo_cage c ON (north_zone.zone = c.zone) INNER JOIN\n\t\tzoo_animal a ON (c.id = a.cage_id)\n\tGROUP BY c.zone, north_zone.squaremeter;"}], 'task-051': [{'correct': False, 'date': 1641657641744, 'query': 'SELECT p.name FROM GHOSTS g INNER JOIN PLACES p ON p.id = g.place_id'}, {'correct': True, 'date': 1641657666159, 'query': 'SELECT p.name FROM GHOSTS g INNER JOIN PLACES p ON p.id = g.place_id WHERE g.name = "Arthur"'}]}
1
{'task-000': [{'correct': True, 'date': 1630314930169, 'query': 'SELECT name from ghosts'}]}
2
{'task-000': [{'correct': True, 'date': 1630583648145, 'q

In [64]:
df_structure= df_structure.astype({"date1":int})

In [65]:
df_structure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5989 entries, 1 to 5989
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   log_id   5989 non-null   object
 1   correct  5989 non-null   object
 2   query    5989 non-null   object
 3   date1    5989 non-null   int64 
 4   task     5989 non-null   object
dtypes: int64(1), object(4)
memory usage: 280.7+ KB


In [66]:
df_structure

,log_id,correct,query,date1,task
1,6123b80df66ddb7127c2abfd,False,from ghosts SELECT name,1629799177614,task-000
2,6123b80df66ddb7127c2abfd,True,SELECT name from ghosts,1629799183760,task-000
3,6123b80df66ddb7127c2abfd,True,SELECT north_zone.squaremeter/count(*) FROM (S...,1641556069420,task-086
4,6123b80df66ddb7127c2abfd,False,SELECT p.name FROM GHOSTS g INNER JOIN PLACES ...,1641657641744,task-051
5,6123b80df66ddb7127c2abfd,True,SELECT p.name FROM GHOSTS g INNER JOIN PLACES ...,1641657666159,task-051
...,...,...,...,...,...
5985,6205117f1c2f8b6e27ef4b2d,False,"SELECT title, name FROM song WHERE language = ...",1644500979984,task-044
5986,6205117f1c2f8b6e27ef4b2d,True,"SELECT title, name FROM song INNER JOIN interp...",1644500994863,task-044
5987,6205117f1c2f8b6e27ef4b2d,False,"SELECT a.title, s.rating, s.title",1644501145215,task-045
5988,6205117f1c2f8b6e27ef4b2d,False,"SELECT a.title, s.rating, s.title FROM song s ...",1644501179493,task-045


In [67]:
#Date1 in Datetime (UTC)
for i in range(1, (len(df_structure)+1)):
    df_structure["date1"][i] = datetime.datetime.fromtimestamp(df_structure["date1"][i] / 1000).strftime('%d/%m/%y %H:%M:%S')

/var/folders/f0/pqy2clvx0nv38xm_npv8fklm0000gn/T/ipykernel_1091/3310210478.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_structure["date1"][i] = datetime.datetime.fromtimestamp(df_structure["date1"][i] / 1000).strftime('%d/%m/%y %H:%M:%S')
/Users/janisfrolich/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [68]:
df_structure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5989 entries, 1 to 5989
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   log_id   5989 non-null   object
 1   correct  5989 non-null   object
 2   query    5989 non-null   object
 3   date1    5989 non-null   object
 4   task     5989 non-null   object
dtypes: object(5)
memory usage: 409.8+ KB


In [69]:
#Verbiungstabelle erstellen
log_id_dict = {}
id = 0
df_structure["log_id"] = 0
for i in range(1, len(df_structure["log_id"])+1):
    id += 1
    df_structure["log_id"][i] = id
    if df_structure["log_id"][i] not in log_id_dict:
        log_id_dict.update({df_structure["log_id"][i]:[id]})
    else:
        log_id_dict[df_structure["log_id"][i]].append(id)

/var/folders/f0/pqy2clvx0nv38xm_npv8fklm0000gn/T/ipykernel_1091/1818635466.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_structure["log_id"][i] = id


In [70]:
#Verbindungstabelle erstellen aus log_id_dict
df_connection = pd.DataFrame(columns=["id", "user_id", "log_id"])
id = 0
for i in log_id_dict.keys():
    for n in log_id_dict[i]:
        new_line = pd.DataFrame({"id": id, "log_id": i, "id_list": n}, index=[id])
        df_connection = df_connection.append(new_line)
        id += 1

In [71]:
df_connection

,id,user_id,log_id,id_list
0,0,NaN,1,1.0
1,1,NaN,2,2.0
2,2,NaN,3,3.0
3,3,NaN,4,4.0
4,4,NaN,5,5.0
...,...,...,...,...
5984,5984,NaN,5985,5985.0
5985,5985,NaN,5986,5986.0
5986,5986,NaN,5987,5987.0
5987,5987,NaN,5988,5988.0


In [72]:
df_structure

,log_id,correct,query,date1,task
1,1,False,from ghosts SELECT name,24/08/21 11:59:37,task-000
2,2,True,SELECT name from ghosts,24/08/21 11:59:43,task-000
3,3,True,SELECT north_zone.squaremeter/count(*) FROM (S...,07/01/22 12:47:49,task-086
4,4,False,SELECT p.name FROM GHOSTS g INNER JOIN PLACES ...,08/01/22 17:00:41,task-051
5,5,True,SELECT p.name FROM GHOSTS g INNER JOIN PLACES ...,08/01/22 17:01:06,task-051
...,...,...,...,...,...
5985,5985,False,"SELECT title, name FROM song WHERE language = ...",10/02/22 14:49:39,task-044
5986,5986,True,"SELECT title, name FROM song INNER JOIN interp...",10/02/22 14:49:54,task-044
5987,5987,False,"SELECT a.title, s.rating, s.title",10/02/22 14:52:25,task-045
5988,5988,False,"SELECT a.title, s.rating, s.title FROM song s ...",10/02/22 14:52:59,task-045


In [73]:
df_connection.to_csv(path_or_buf="df_connection.csv", sep=";")

In [74]:
df_structure.to_csv(path_or_buf="log_data.csv", sep=",")